# <center/>MindInsight的溯源分析和对比分析体验

[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/notebook/mindinsight/mindspore_lineage_and_scalars_comparison.ipynb)

## 概述

在模型调参的场景下，需要多次调整模型超参并进行多次训练，这个过程，往往需要手动记录每次训练使用参数以及训练结果。为此，MindSpore提供了自动记录模型参数、训练信息及训练结果评估指标的功能，并通过MindInsight进行可视化展示。本次体验会从MindInsight的数据记录、可视化效果、如何方便用户在模型调优和数据调优上做一次整体流程的体验。

下面按照MindSpore的训练数据模型的正常步骤进行，使用`SummaryCollector`进行数据保存操作，本次体验的整体流程如下：

1. 数据集的准备，这里使用的是MNIST数据集。

2. 构建一个网络，这里使用LeNet网络。

3. 记录数据及启动训练。

4. 启动MindInsight服务。

5. 溯源分析的使用。

6. 对比分析的使用。

> 本文档适用于GPU和Ascend环境。

## 数据集准备

### 数据集下载

在Jupyter Notebook中执行如下命令，进行数据集下载并解压，将解压后的数据集移动到指定位置。

In [ ]:
!mkdir -p ./datasets/MNIST_Data/train ./datasets/MNIST_Data/test
!wget -NP ./datasets/MNIST_Data/train https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-labels-idx1-ubyte --no-check-certificate
!wget -NP ./datasets/MNIST_Data/train https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/train-images-idx3-ubyte --no-check-certificate
!wget -NP ./datasets/MNIST_Data/test https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-labels-idx1-ubyte --no-check-certificate
!wget -NP ./datasets/MNIST_Data/test https://mindspore-website.obs.myhuaweicloud.com/notebook/datasets/mnist/t10k-images-idx3-ubyte --no-check-certificate

解压后的数据集文件目录结构如下：

```text
./datasets/MNIST_Data
├── test
│   ├── t10k-images-idx3-ubyte
│   └── t10k-labels-idx1-ubyte
└── train
    ├── train-images-idx3-ubyte
    └── train-labels-idx1-ubyte
```

### 数据集处理

数据集处理对于训练非常重要，好的数据集可以有效提高训练精度和效率。在加载数据集前，我们通常会对数据集进行一些处理。
<br/>我们定义一个函数`create_dataset`来创建数据集。在这个函数中，我们定义好需要进行的数据增强和处理操作：

1. 定义数据集。
2. 定义进行数据增强和处理所需要的一些参数。
3. 根据参数，生成对应的数据增强操作。
4. 使用`map`映射函数，将数据操作应用到数据集。
5. 对生成的数据集进行处理。

具体的数据集操作可以在MindInsight的数据溯源中进行可视化分析。

In [2]:
import mindspore.dataset.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.vision import Inter
from mindspore import dtype as mstype
import mindspore.dataset as ds

def create_dataset(data_path, batch_size=16, repeat_size=1,
                   num_parallel_workers=1):
    """ create dataset for train or test
    Args:
        data_path (str): Data path
        batch_size (int): The number of data records in each group
        repeat_size (int): The number of replicated data records
        num_parallel_workers (int): The number of parallel workers
    """
    # define dataset
    mnist_ds = ds.MnistDataset(data_path)

    # define some parameters needed for data enhancement and rough justification
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # according to the parameters, generate the corresponding data enhancement method
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
    # if you need to use SummaryCollector to extract image data, do not use the following normalize operator operation
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW()
    type_cast_op = C.TypeCast(mstype.int32)

    # using map method to apply operations to a dataset
    mnist_ds = mnist_ds.map(operations=type_cast_op, input_columns="label", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=resize_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=rescale_nml_op, input_columns="image", num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(operations=hwc2chw_op, input_columns="image", num_parallel_workers=num_parallel_workers)

    # process the generated dataset
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)  # 10000 as in LeNet train script
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

## 定义LeNet5网络

本例采用的网络模型为LeNet5网络，对于手写数字分类表现得非常出色，网络模型定义如下。

In [3]:
import mindspore.ops as ops
import mindspore.nn as nn
from mindspore.common.initializer import Normal

class LeNet5(nn.Cell):
    """Lenet network structure."""
    def __init__(self):
        super(LeNet5, self).__init__()
        self.batch_size = 32 
        self.conv1 = nn.Conv2d(1, 6, 5, pad_mode="valid")
        self.conv2 = nn.Conv2d(6, 16, 5, pad_mode="valid")
        self.fc1 = nn.Dense(16 * 5 * 5, 120, Normal(0.02), Normal(0.02))
        self.fc2 = nn.Dense(120, 84, Normal(0.02), Normal(0.02))
        self.fc3 = nn.Dense(84, 10)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.image_summary = ops.ImageSummary()
        self.tensor_summary = ops.TensorSummary()

    def construct(self, x):
        self.image_summary("image", x)
        self.tensor_summary("tensor", x)
        x = self.max_pool2d(self.relu(self.conv1(x)))
        x = self.max_pool2d(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 记录数据及启动训练

MindSpore 提供 `SummaryCollector` 接口来记录训练过程中的信息。

为了更好的体验溯源分析和对比分析的效果，这里将调整学习率（`learning_rate`）、迭代次数（`epoch_size`）、batch数量（`batch_size`）来多次训练模型，并使用`SummaryCollector`保存对应的数据。

`learning_rate`取值分别为0.01和0.05。

`epoch_size`取值分别为2和5。

`batch_size`取值分别为16和32。

每次调整一个参数进行训练，总共分2x2x2=8组参数。

`SummaryCollector`的更多用法，请参考[API文档](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.train.html#mindspore.train.callback.SummaryCollector)。

In [4]:
from mindspore.train.callback import SummaryCollector
from mindspore.nn import Accuracy
from mindspore import context, Model
from mindspore.nn import SoftmaxCrossEntropyWithLogits
from mindspore import load_checkpoint, load_param_into_net
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor
import os

if __name__=="__main__":
    context.set_context(mode=context.GRAPH_MODE, device_target = "GPU")
    if os.name == "nt":
        os.system("del/f/s/q *.ckpt *.meta")
    else:
        os.system("rm -f *.ckpt *.meta *.pb")

    mnist_path = "./datasets/MNIST_Data/"
    model_path = "./models/ckpt/lineage_and_scalars_comparison/"
    repeat_size = 1
    config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)
    ckpoint_cb = ModelCheckpoint(prefix="checkpoint_lenet", directory=model_path, config=config_ck)
    # define the optimizer

    lrs = [0.01,0.05]
    epoch_sizes = [2, 5]
    batch_sizes = [16, 32]
    situations = [(i, j, k) for i in lrs for j in epoch_sizes for k in batch_sizes]
    count = 1

    for lr,epoch_size,batch_size in situations:
        momentum = 0.9 
        network = LeNet5()
        net_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
        net_opt = nn.Momentum(network.trainable_params(), lr, momentum)
        model = Model(network, net_loss, net_opt, metrics={"Accuracy": Accuracy()})
        summary_collector = SummaryCollector(summary_dir="./summary_base/LeNet-MNIST_Data,lr:{},epoch:{},batch_size:{}"
                                             .format(lr, epoch_size, batch_size), collect_freq=1)
        # Start to train
        print("================= The Situation {} =================".format(count))
        print("== learning_rate:{}, epoch_size:{}, batch_size:{} ==".format(lr, epoch_size, batch_size))
        print("================ Starting Training ================")
        ds_train = create_dataset(os.path.join(mnist_path, "train"), batch_size, repeat_size)
        model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, summary_collector, LossMonitor(125)], dataset_sink_mode=True)

        print("================ Starting Testing ================")
        # load testing dataset
        ds_eval = create_dataset(os.path.join(mnist_path, "test"))
        acc = model.eval(ds_eval, callbacks=[summary_collector], dataset_sink_mode=True)
        print("============ Accuracy:{} ============\n\n".format(acc))
        count += 1

================= The Situation 1 =================
== learning_rate:0.01, epoch_size:2, batch_size:16 ==
================ Starting Training ================
epoch: 1 step: 125, loss is 2.296706
epoch: 1 step: 250, loss is 2.2627764
epoch: 1 step: 375, loss is 2.3244872
epoch: 1 step: 500, loss is 2.3250148
epoch: 1 step: 625, loss is 2.2620986
epoch: 1 step: 750, loss is 2.3121898
epoch: 1 step: 875, loss is 2.3026366
epoch: 1 step: 1000, loss is 2.2881525
epoch: 1 step: 1125, loss is 2.3082426
epoch: 1 step: 1250, loss is 2.2710335
epoch: 1 step: 1375, loss is 2.270731
epoch: 1 step: 1500, loss is 0.9243496
epoch: 1 step: 1625, loss is 0.6205256
epoch: 1 step: 1750, loss is 0.24102339
epoch: 1 step: 1875, loss is 0.22378448
epoch: 1 step: 2000, loss is 0.29994896
epoch: 1 step: 2125, loss is 0.17046359
epoch: 1 step: 2250, loss is 0.0100224735
epoch: 1 step: 2375, loss is 0.48139822
epoch: 1 step: 2500, loss is 0.06742601
epoch: 1 step: 2625, loss is 0.0052048573
epoch: 1 step: 2750,

## 启动及关闭MindInsight服务

这里主要展示如何启用及关闭MindInsight，更多的命令集信息，请参考MindSpore官方网站：<https://www.mindspore.cn/mindinsight/docs/zh-CN/master/index.html>。

启动MindInsight服务命令：

```
mindinsight start --summary-base-dir=./summary_base --port=8080
```

- `--summary-base-dir`：MindInsight指定启动工作路径的命令；`./summary_base` 为 `SummaryCollector` 的 `summary_dir` 参数所指定的目录。
- `--port`：MindInsight指定启动的端口，数值可以任意为1~65535的范围内。

停止MindInsight服务命令：

```
mindinsight stop --port=8080
```

- `mindinsight stop`：MindInsight关闭服务命令。
- `--port=8080`：即MindInsight服务开启在`8080`端口，所以这里写成`--port=8080`。

## 溯源分析

### 连接到溯源分析地址

浏览器中输入:`http://127.0.0.1:8080`进入MindInsight界面如下：

![image](https://gitee.com/mindspore/docs/raw/master/docs/notebook/mindinsight/images/mindinsight_homepage_for_lineage.png)

### 模型溯源界面介绍

上图训练列表中序号1-8分别是按照8组训练参数，保存的训练数据。点击右上角的溯源分析便可以进入，溯源分析包含模型溯源和数据溯源，首先是模型溯源界面，如下所示：

![image](https://gitee.com/mindspore/docs/raw/master/docs/notebook/mindinsight/images/model_lineage_page.png)

#### 优化目标区域

可以选择模型精度值（Accuracy）或模型损失值（loss）。

![image](https://gitee.com/mindspore/docs/raw/master/docs/notebook/mindinsight/images/optimization_target_page_of_model_lineage.png)

能直观的看出`learning_rate`、`epoch`、`batch_size`三个参数对本次训练模型的精度值和损失值的参数重要性（参数重要性的数值越接近1表示对此优化目标的影响越大，越接近0则表示对优化目标的影响越小），方便用户决策在训练时需要调整的参数。

#### 模型训练的详细参数展示界面

展示界面中提供了模型训练过程中的各类重要参数信息，包括：网络、优化器、训练样本数量、测试样本数量、学习率、迭代次数、`batch_size`、`device`数目、模型大小、损失函数等等，用户可以自行选择单次训练数据进行可视化分析或者多次训练数据进行可视化比对分析，提高分析效率。

![image](https://gitee.com/mindspore/docs/raw/master/docs/notebook/mindinsight/images/detailed_information_page_of_model_lineage.png)

### 数据溯源界面介绍

数据溯源展示了用户进行模型训练前的数据增强的过程，且此过程按照增强顺序进行排列。

![image](https://gitee.com/mindspore/docs/raw/master/docs/notebook/mindinsight/images/data_lineage_page.png)

本例中数据增强的过程包括`MnistDataset`，`Map_TypeCast`，`Map_Resize`，`Map_Rescale`，`Map_HWC2CHW`，`Shuffle`，`Batch`等操作。

- 数据集转换（`MnistDataset`）
- label的数据类型转换（`Map_TypeCast`）
- 图像的高宽缩放（`Map_Resize`）
- 图像的比例缩放（`Map_Rescale`）
- 图像数据的张量变换（`Map_HWC2CHW`）
- 图像混洗（`Shuffle`）
- 图像成组（`Batch`）

## 对比分析

### 进入对比分析界面

从MindInsight主页进入对比分析界面。

![image](https://gitee.com/mindspore/docs/raw/master/docs/notebook/mindinsight/images/mindinsight_homepage_for_scalars_comparison.png)

从对比分析界面中可以对比不同的训练中的标量信息，本例使用`SummaryCollector`自动保存了loss值，其他的标量信息保存，请参考[官方文档](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/ops/mindspore.ops.ScalarSummary.html)。

![image](https://gitee.com/mindspore/docs/raw/master/docs/notebook/mindinsight/images/scalars_comparison_page.png)

对比看板中可以选择对比的信息有：

- 训练选择：本例有8组不同的训练参数对应的训练信息可供选择，此次选择了其中学习率（lr）分别为0.01和0.05的两组训练过程的数据进行对比。
- 标签选择：本例保存了loss值一种标量标签。

> 对比曲线可通过调整平滑度来优化显示效果。

## 总结

本次体验使用了MindSpore的数据收集接口`SummaryCollector`对不同训练参数下的模型训练信息进行收集，并且通过开启MindInsight服务将溯源信息和标量信息进行可视化展示，以上就是本次体验的全部内容。